In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split

df = pd.read_csv(r'/Users/viniciustormin/02 Areas/UFG/MD/dataset/UCMF_limpo.csv')
df.head()

,Peso,Altura,IDADE,PULSOS,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,SEXO,MOTIVO,HDA,IMC,IMC_CATEGORIA_SIMPLIFICADA
0,5.0,51.0,0.114990,Normais,Não Calculado,Anormal,Normal,Sistólico,112.0,M,palpitação/taquicardia/arritmia,palpitacao,19.223376,Normal
1,3.5,50.0,0.016427,Normais,Não Calculado,Anormal,Normal,ausente,128.0,M,dispnéia,dispneia,14.000000,Baixo
2,16.0,99.0,3.819302,Normais,Não Calculado,Anormal,Normal,Sistólico,88.0,M,check-up,assintomático,16.324865,Baixo
3,8.1,65.0,0.484600,Normais,Não Calculado,Anormal,Normal,ausente,92.0,M,parecer cardiológico,assintomático,19.171598,Normal
4,40.0,151.0,12.418891,Normais,Não Calculado,Anormal,Normal,ausente,96.0,M,parecer cardiológico,dor precordial,17.543090,Baixo


In [18]:
df['NORMAL X ANORMAL'].value_counts()

NORMAL X ANORMAL
Normal     6575
Anormal    4935
anormal       1
Normais       1
Name: count, dtype: int64

In [19]:
def z_score_group(x):
    return (x - x.mean()) / x.std()

bins = [-1, 2, 5, 12, 20]
labels = ['Bebê (0-2)', 'Pre-Escolar (2-5)', 'Escolar (5-12)', 'Adolescente (12+)']
df['Faixa_Etaria'] = pd.cut(df['IDADE'], bins=bins, labels=labels)

df['IMC_Z_Score_Proxy'] = df.groupby('Faixa_Etaria')['IMC'].transform(z_score_group)
df['FC_Z_Score_Proxy'] = df.groupby('Faixa_Etaria')['FC'].transform(z_score_group)

/var/folders/7g/m73zdvk573j9syqps150mbbc0000gn/T/ipykernel_62441/2368749788.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['IMC_Z_Score_Proxy'] = df.groupby('Faixa_Etaria')['IMC'].transform(z_score_group)
/var/folders/7g/m73zdvk573j9syqps150mbbc0000gn/T/ipykernel_62441/2368749788.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['FC_Z_Score_Proxy'] = df.groupby('Faixa_Etaria')['FC'].transform(z_score_group)


In [20]:
df['HDA'].isna().sum()
df['HDA'].value_counts()

HDA
assintomático                     6519
dor precordial                     406
dispneia                           396
palpitacao                         305
cianose                            195
desmaio/tontura                    124
outro                              121
ganho de peso                       79
cianose,dispneia                    74
dor precordial,palpitacao           71
dor precordial,dispneia             62
cianose,desmaio/tontura             27
palpitacao,dispneia                 26
outro,dispneia                      16
dor precordial,desmaio/tontura      15
dispneia,palpitacao                 14
ganho de peso,dispneia              14
desmaio/tontura,palpitacao          11
cianose,palpitacao                   8
outro,desmaio/tontura                8
dor precordial,cianose               7
desmaio/tontura,dispneia             7
dor precordial,ganho de peso         6
dor precordial,outro                 5
dispneia,desmaio/tontura             5
outro,cianose        

In [21]:
df['HDA'] = df['HDA'].fillna('assintomático')

In [22]:
df['HDA_lower'] = df['HDA'].str.lower()

df['Sintoma_Dor'] = df['HDA_lower'].str.contains('dor').astype(int)
df['Sintoma_Cianose'] = df['HDA_lower'].str.contains('cianose').astype(int)
df['Sintoma_Dispn'] = df['HDA_lower'].str.contains('dispneia|dispnéia').astype(int)
df['Checkup_Assintomatico'] = df['HDA_lower'].str.contains('assintomático|check-up').astype(int)

termos_hipertensao = ['Hipertensão', 'Elevada', 'HAS']
df['PPA_Alterada'] = df['PPA'].apply(lambda x: 1 if any(t in str(x) for t in termos_hipertensao) else 0)

df.head()

,Peso,Altura,IDADE,PULSOS,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,SEXO,...,IMC_CATEGORIA_SIMPLIFICADA,Faixa_Etaria,IMC_Z_Score_Proxy,FC_Z_Score_Proxy,HDA_lower,Sintoma_Dor,Sintoma_Cianose,Sintoma_Dispn,Checkup_Assintomatico,PPA_Alterada
0,5.0,51.0,0.114990,Normais,Não Calculado,Anormal,Normal,Sistólico,112.0,M,...,Normal,Bebê (0-2),0.460415,0.099711,palpitacao,0,0,0,0,0
1,3.5,50.0,0.016427,Normais,Não Calculado,Anormal,Normal,ausente,128.0,M,...,Baixo,Bebê (0-2),-0.263540,0.965478,dispneia,0,0,1,0,0
2,16.0,99.0,3.819302,Normais,Não Calculado,Anormal,Normal,Sistólico,88.0,M,...,Baixo,Pre-Escolar (2-5),-0.105394,-0.447359,assintomático,0,0,0,1,0
3,8.1,65.0,0.484600,Normais,Não Calculado,Anormal,Normal,ausente,92.0,M,...,Normal,Bebê (0-2),0.453239,-0.982498,assintomático,0,0,0,1,0
4,40.0,151.0,12.418891,Normais,Não Calculado,Anormal,Normal,ausente,96.0,M,...,Baixo,Adolescente (12+),-0.259982,1.618287,dor precordial,1,0,0,0,0


In [23]:
df['MOTIVO_lower'] = df['MOTIVO'].str.lower()

df['Motivo_Cirurgia'] = df['MOTIVO_lower'].str.contains('cirurgia').astype(int)
df['Motivo_Sopro'] = df['MOTIVO_lower'].str.contains('sopro').astype(int)
df['Motivo_Congenita'] = df['MOTIVO_lower'].str.contains('congenita|congênita').astype(int)


In [24]:
df.head()

,Peso,Altura,IDADE,PULSOS,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,SEXO,...,HDA_lower,Sintoma_Dor,Sintoma_Cianose,Sintoma_Dispn,Checkup_Assintomatico,PPA_Alterada,MOTIVO_lower,Motivo_Cirurgia,Motivo_Sopro,Motivo_Congenita
0,5.0,51.0,0.114990,Normais,Não Calculado,Anormal,Normal,Sistólico,112.0,M,...,palpitacao,0,0,0,0,0,palpitação/taquicardia/arritmia,0,0,0
1,3.5,50.0,0.016427,Normais,Não Calculado,Anormal,Normal,ausente,128.0,M,...,dispneia,0,0,1,0,0,dispnéia,0,0,0
2,16.0,99.0,3.819302,Normais,Não Calculado,Anormal,Normal,Sistólico,88.0,M,...,assintomático,0,0,0,1,0,check-up,0,0,0
3,8.1,65.0,0.484600,Normais,Não Calculado,Anormal,Normal,ausente,92.0,M,...,assintomático,0,0,0,1,0,parecer cardiológico,0,0,0
4,40.0,151.0,12.418891,Normais,Não Calculado,Anormal,Normal,ausente,96.0,M,...,dor precordial,1,0,0,0,0,parecer cardiológico,0,0,0


In [25]:
correcoes = {
    'anormal': 'Anormal',
    'Normais': 'Normal',
    'Anormal': 'Anormal',
    'Normal': 'Normal'
}
df['NORMAL X ANORMAL'] = df['NORMAL X ANORMAL'].map(correcoes)

df['Target'] = df['NORMAL X ANORMAL'].map({'Anormal': 1, 'Normal': 0})

print(df['Target'].value_counts())

df = df.drop(columns=['NORMAL X ANORMAL'])

Target
0    6576
1    4936
Name: count, dtype: int64


In [26]:
mapa_sopro = {
    'sistólico': 'Sistolico',  
    'Sistólico': 'Sistolico',  
    'ausente': 'Ausente',      
    'Não Informado': 'Ausente',
    'contínuo': 'Continuo',
    'Contínuo': 'Continuo',
    'diastólico': 'Diastolico',
    'Sistolico e diastólico': 'Sistolico_Diastolico' 
}

df['SOPRO'] = df['SOPRO'].map(mapa_sopro).fillna('Ausente')

print("Valores únicos após limpeza:")
print(df['SOPRO'].value_counts())

Valores únicos após limpeza:
SOPRO
Ausente                 7103
Sistolico               4351
Continuo                  47
Diastolico                 8
Sistolico_Diastolico       3
Name: count, dtype: int64


In [27]:
df['SOPRO'] = df['SOPRO'].replace('Não Informado', 'ausente')

df = pd.get_dummies(df, columns=['SOPRO'], prefix='SOPRO')

cols_sopro = [c for c in df.columns if 'SOPRO_' in c]
df[cols_sopro] = df[cols_sopro].astype(int)

In [28]:
mapa_b2 = {
    'Normal': 'Normal',
    'Hiperfonética': 'Hiperfonetica', 
    'Desdob fixo': 'Desdobramento_Fixo', 
    'Única': 'Unica', 
    'Outro': 'Outro'
}
df['B2'] = df['B2'].map(mapa_b2).fillna('Outro')

df = pd.get_dummies(df, columns=['B2'], prefix='B2')

cols_b2 = [c for c in df.columns if 'B2_' in c]
df[cols_b2] = df[cols_b2].astype(int)

In [29]:
cols_to_drop_now = [
    'Peso', 'Altura', 'PULSOS', 
    'HDA', 'HDA_lower', 
    'MOTIVO', 'MOTIVO_lower', 
    'PPA'
]

df_cleaned = df.drop(columns=cols_to_drop_now)
df_cleaned.columns

Index(['IDADE', 'FC', 'SEXO', 'IMC', 'IMC_CATEGORIA_SIMPLIFICADA',
       'Faixa_Etaria', 'IMC_Z_Score_Proxy', 'FC_Z_Score_Proxy', 'Sintoma_Dor',
       'Sintoma_Cianose', 'Sintoma_Dispn', 'Checkup_Assintomatico',
       'PPA_Alterada', 'Motivo_Cirurgia', 'Motivo_Sopro', 'Motivo_Congenita',
       'Target', 'SOPRO_Ausente', 'SOPRO_Continuo', 'SOPRO_Diastolico',
       'SOPRO_Sistolico', 'SOPRO_Sistolico_Diastolico',
       'B2_Desdobramento_Fixo', 'B2_Hiperfonetica', 'B2_Normal', 'B2_Outro',
       'B2_Unica'],
      dtype='object')

In [35]:
df_cleaned.to_csv(r'/Users/viniciustormin/02 Areas/UFG/MD/dataset/UCMF_eng_feature.csv', index=False)


In [33]:
# One-hot encoding para IMC_CATEGORIA_SIMPLIFICADA
df_cleaned = pd.get_dummies(df_cleaned, columns=['IMC_CATEGORIA_SIMPLIFICADA'], prefix='IMC_Cat')

# Converte para int (pd.get_dummies retorna bool por padrão)
cols_imc_cat = [c for c in df_cleaned.columns if 'IMC_Cat_' in c]
df_cleaned[cols_imc_cat] = df_cleaned[cols_imc_cat].astype(int)
print("Colunas criadas:")
print(cols_imc_cat)
df_cleaned[cols_imc_cat].head()

KeyError: "None of [Index(['IMC_CATEGORIA_SIMPLIFICADA'], dtype='object')] are in the [columns]"

In [34]:
df_cleaned.head()

,IDADE,FC,SEXO,IMC,Faixa_Etaria,IMC_Z_Score_Proxy,FC_Z_Score_Proxy,Sintoma_Dor,Sintoma_Cianose,Sintoma_Dispn,...,SOPRO_Sistolico_Diastolico,B2_Desdobramento_Fixo,B2_Hiperfonetica,B2_Normal,B2_Outro,B2_Unica,IMC_Cat_Baixo,IMC_Cat_Normal,IMC_Cat_Obesidade,IMC_Cat_Sobrepeso
0,0.114990,112.0,M,19.223376,Bebê (0-2),0.460415,0.099711,0,0,0,...,0,0,0,1,0,0,0,1,0,0
1,0.016427,128.0,M,14.000000,Bebê (0-2),-0.263540,0.965478,0,0,1,...,0,0,0,1,0,0,1,0,0,0
2,3.819302,88.0,M,16.324865,Pre-Escolar (2-5),-0.105394,-0.447359,0,0,0,...,0,0,0,1,0,0,1,0,0,0
3,0.484600,92.0,M,19.171598,Bebê (0-2),0.453239,-0.982498,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,12.418891,96.0,M,17.543090,Adolescente (12+),-0.259982,1.618287,1,0,0,...,0,0,0,1,0,0,1,0,0,0


In [36]:
X = df_cleaned.drop(columns=['Target'])
y = df_cleaned['Target']

In [37]:
print("Colunas que ficaram no X (para divisão):")
print(X.columns.tolist())

Colunas que ficaram no X (para divisão):
['IDADE', 'FC', 'SEXO', 'IMC', 'Faixa_Etaria', 'IMC_Z_Score_Proxy', 'FC_Z_Score_Proxy', 'Sintoma_Dor', 'Sintoma_Cianose', 'Sintoma_Dispn', 'Checkup_Assintomatico', 'PPA_Alterada', 'Motivo_Cirurgia', 'Motivo_Sopro', 'Motivo_Congenita', 'SOPRO_Ausente', 'SOPRO_Continuo', 'SOPRO_Diastolico', 'SOPRO_Sistolico', 'SOPRO_Sistolico_Diastolico', 'B2_Desdobramento_Fixo', 'B2_Hiperfonetica', 'B2_Normal', 'B2_Outro', 'B2_Unica', 'IMC_Cat_Baixo', 'IMC_Cat_Normal', 'IMC_Cat_Obesidade', 'IMC_Cat_Sobrepeso']


In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [39]:
def atribuir_faixa_etaria(df_input):
    bins = [-1, 2, 5, 12, 20, 100]
    labels = ['0-2', '2-5', '5-12', '12-20', '20+']
    return pd.cut(df_input['IDADE'], bins=bins, labels=labels)

X_train['Faixa_Etaria'] = atribuir_faixa_etaria(X_train)
X_test['Faixa_Etaria'] = atribuir_faixa_etaria(X_test)

referencia_imc = X_train.groupby('Faixa_Etaria')['IMC'].agg(['mean', 'std'])
referencia_fc = X_train.groupby('Faixa_Etaria')['FC'].agg(['mean', 'std'])

print("Tabela de Referência aprendida (Treino):")
print(referencia_imc)

Tabela de Referência aprendida (Treino):
                   mean        std
Faixa_Etaria                      
0-2           15.923058   7.304960
2-5           16.923390   5.737442
5-12          20.327031  25.537839
12-20         24.993803  29.300033
20+           24.630354   7.887280


/var/folders/7g/m73zdvk573j9syqps150mbbc0000gn/T/ipykernel_62441/692263683.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  referencia_imc = X_train.groupby('Faixa_Etaria')['IMC'].agg(['mean', 'std'])
/var/folders/7g/m73zdvk573j9syqps150mbbc0000gn/T/ipykernel_62441/692263683.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  referencia_fc = X_train.groupby('Faixa_Etaria')['FC'].agg(['mean', 'std'])


In [40]:
def calcular_z_com_referencia(row, tabela_ref, col_valor):
    faixa = row['Faixa_Etaria']
    
    if pd.isna(faixa):
        return 0
        
    if faixa not in tabela_ref.index:
        return 0
        
    media = tabela_ref.loc[faixa, 'mean']
    desvio = tabela_ref.loc[faixa, 'std']
    
    if desvio == 0: return 0
    return (row[col_valor] - media) / desvio

X_train['IMC_Z'] = X_train.apply(lambda x: calcular_z_com_referencia(x, referencia_imc, 'IMC'), axis=1)
X_train['FC_Z'] = X_train.apply(lambda x: calcular_z_com_referencia(x, referencia_fc, 'FC'), axis=1)

X_test['IMC_Z'] = X_test.apply(lambda x: calcular_z_com_referencia(x, referencia_imc, 'IMC'), axis=1)
X_test['FC_Z'] = X_test.apply(lambda x: calcular_z_com_referencia(x, referencia_fc, 'FC'), axis=1)

cols_auxiliares = ['IMC', 'FC', 'IDADE', 'Faixa_Etaria'] 
X_train = X_train.drop(columns=['IMC', 'FC', 'Faixa_Etaria'])
X_test = X_test.drop(columns=['IMC', 'FC', 'Faixa_Etaria'])

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [41]:
X_train.to_csv(r'/Users/viniciustormin/02 Areas/UFG/MD/dataset/X_train.csv', index=False)
X_test.to_csv(r'/Users/viniciustormin/02 Areas/UFG/MD/dataset/X_test.csv', index=False)
y_train.to_csv(r'/Users/viniciustormin/02 Areas/UFG/MD/dataset/y_train.csv', index=False)
y_test.to_csv(r'/Users/viniciustormin/02 Areas/UFG/MD/dataset/y_test.csv', index=False)